#  aitextgen — Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: May 16th, 2021 (aitextgen v0.5.2)*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [ ]:
!pip install -q aitextgen

     |████████████████████████████████| 572 kB 11.4 MB/s 
     |████████████████████████████████| 4.2 MB 19.7 MB/s 
     |████████████████████████████████| 87 kB 4.2 MB/s 
     |████████████████████████████████| 585 kB 12.1 MB/s 
     |████████████████████████████████| 418 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
     |████████████████████████████████| 140 kB 20.9 MB/s 
     |████████████████████████████████| 1.1 MB 57.1 MB/s 
     |████████████████████████████████| 6.6 MB 54.9 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 144 kB 13.1 MB/s 
     |████████████████████████████████| 271 kB 62.7 MB/s 


In [ ]:
import aitextgen
import datetime
import gc
import logging
import os
import requests
import torch

In [ ]:
session_url = 'http://172.28.0.2:9000/api/sessions'
notebook_name = requests.get(session_url).json()[0]['name']

run_datetime = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
run_id = notebook_name + '_run_' + run_datetime

In [ ]:
log_format = '%(asctime)s — %(levelname)s — %(name)s — %(message)s'
date_format = '%d/%m/%Y %H:%M:%S'
log_level = logging.DEBUG

logging.basicConfig(format=log_format, datefmt=date_format, level=log_level)

## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, an Nvidia P100, or an Nvidia V100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM. **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [ ]:
!nvidia-smi

Tue Jun  7 19:03:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
aitextgen.colab.mount_gdrive()

Mounted at /content/drive


In [ ]:
gdrive_root_dir = '/content/drive/My Drive'

## Loading GPT-2 or GPT Neo

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M` (default): the "medium" model, 1.5GB on disk.
* `774M` (default): the "large" model, 3GB on disk.

You can also finetune a GPT Neo model instead, which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model.
* `350M`: Analogous to the GPT-2 355M model

The next cell downloads the model and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [ ]:
load_model = None
#load_model = 'aitextgen-CCS-124M-7200-3600_run_2022-05-06-18-01-15'

In [ ]:
#model='124M'
#model='355M'
#model='774M'

model='gpt-neo-125M'
#model='gpt-neo-350M'

In [ ]:
if load_model is None:
    if model == '124M' or model == '355M' or model == '774M':
        ai = aitextgen.aitextgen(tf_gpt2=model, to_gpu=True)
    else:
        ai = aitextgen.aitextgen(model='EleutherAI/' + model, to_gpu=True)

07/06/2022 19:04:01 — INFO — aitextgen — Downloading EleutherAI/gpt-neo-125M model to /aitextgen.
07/06/2022 19:04:01 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
07/06/2022 19:04:01 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json not found in cache or force_download set to True, downloading to /content/aitextgen/tmp5zj49n58
07/06/2022 19:04:01 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
07/06/2022 19:04:01 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "GET /EleutherAI/gpt-neo-125M/resolve/main/config.json HTTP/1.1" 200 1007


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json in cache at aitextgen/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
creating metadata file for aitextgen/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
07/06/2022 19:04:01 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
07/06/2022 19:04:01 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/pytorch_model.bin not found in cache or force_download set to True, downloading to /content/aitextgen/tmplerjmogy
07/06/2022 19:04:01 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): cdn-lfs.huggingface.co:443
07/06/2022 19:04:01 — DEBUG — urllib3.con

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/pytorch_model.bin in cache at aitextgen/b0ace3b93ace62067a246888f1e54e2d3ec20807d4d3e27ac602eef3b7091c0b.6525df88f1d5a2d33d95ce2458ef6af9658fe7d1393d6707e0e318779ccc68ff
creating metadata file for aitextgen/b0ace3b93ace62067a246888f1e54e2d3ec20807d4d3e27ac602eef3b7091c0b.6525df88f1d5a2d33d95ce2458ef6af9658fe7d1393d6707e0e318779ccc68ff
07/06/2022 19:04:12 — INFO — aitextgen — Using the tokenizer for EleutherAI/gpt-neo-125M.
07/06/2022 19:04:12 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
07/06/2022 19:04:12 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /content/aitextgen/tmp2m0oi6xm
07/06/2022 19:04:13 — DEBUG — urllib3.connectionpool —

...

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/merges.txt in cache at aitextgen/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for aitextgen/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
07/06/2022 19:04:13 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
07/06/2022 19:04:13 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/tokenizer.json HTTP/1.1" 404 0
07/06/2022 19:04:13 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
07/06/2022 19:04:13 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/added_tokens.json HTTP/1.1" 404 0
07/06/2022 19:04:13 — DEBUG — urllib3.connectionpool — Starting new

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/special_tokens_map.json in cache at aitextgen/6c3239a63aaf46ec7625b38abfe41fc2ce0b25f90800aefe6526256340d4ab6d.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c
creating metadata file for aitextgen/6c3239a63aaf46ec7625b38abfe41fc2ce0b25f90800aefe6526256340d4ab6d.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c
07/06/2022 19:04:13 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
07/06/2022 19:04:13 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/vocab.json from cache at aitextgen/08c00c4159e921d4c941ac75732643373aba509d9b352a82bbbb043a94058d98.a552555fdda56a1c7c9a285bccfd44ac8e4b9e26c8c9b307831b3ea3ac782b45
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/merge

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
creating metadata file for /root/.cache/huggingface/transformers/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
07/06/2022 19:04:14 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
07/06/2022 19:04:14 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/config.json HTTP/1.1" 200 0
07/06/2022 19:04:14 — INFO — aitextgen — GPTNeo loaded with 125M parameters.


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

**If you just trained a model**, you'll get much faster training performance if you reload the model; the next cell will reload the model you just trained from the `trained_model` folder.

`generate()` without any parameters generates a single text from the loaded model to the console.

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
num_outputs = 10
max_length = 1000
temperature = 1.0
top_p = 0.9

prompts = ['',
           'APT41 is a ',
           'APT41 is a state-sponsored espionage group ',
           'Digital Forensics ',
           'Digital Forensics Analysis Report\n',
           'This report is ',
           'The contents of ',
           'Conclusion\n',
           'It is recommended that ',
           'In the opinion of the expert, ',
           'File \'Exploit_Office\' contains ',
           'File \'Exploit_Office\' does not contain ',
           'Website \'Webmail SquirrelMail\' contains ',
           'Website \'Webmail SquirrelMail\' does not contain ',
           'Bill Due to past contains a link \'https://genom.mefst.hr/webmail/src/login.php\' to a website \'Webmail SquirrelMail\'.',
           'New Dogecoin Crypto Sale contains a link \'http://webmail.forumofthemall.hr/mail/loging.php\' to a website \'Webmail SquirrelMail Popular Forum\'.',
           'New OneCoin Crypto Sale contains a link \'http://',
           'Note of eviction contains ',
           'Note of eviction contains attachment ',
           'Note of eviction contains attachment \'Exploit_Office\'. Attachment is quarantined on \'Mail server EP\'.',
           'Log entry found: ',
           'Log entry found: Firewall (Type: Firewall) ',
           'Log entry found: Firewall (Type: Firewall) detected. [Allowed network traffic protocol ',
           'Log entry found: Firewall (Type: Firewall) blocked. [Blocked network traffic protocol ',
           'Log entry found: Firewall (Type: Firewall) detected. [Allowed network traffic protocol \'smtp:25\' from \'server74.aws.com\' to \'Mail server EP\'. Rule \'Internet_to_Mail_Server\'.]',
           'Log entry found: Firewall (Type: Firewall) detected. [Allowed network traffic protocol \'https:443\' from \'Proxy server\' to \'server74.aws.com\'. Rule \'Proxy_to_Internet, https:443\'.]',
           'Log entry found: Firewall (Type: Firewall) blocked. [Blocked network traffic protocol \'https:443\' from \'PCSZT03\' to \'Firewall TSO Enterprise\'.]',
           'Log analysis on ',
           'Log analysis on \'Firewall TSO Enterprise\' for period 1.1.2022. 0:00:00 - 4.2.2022. 13:50:44 finished. Report is ready.']

In [ ]:
output_root_dir = gdrive_root_dir + '/aitextgen/outputs/' + run_id
output_ext = '.txt'

if not os.path.exists(output_root_dir):
    os.makedirs(output_root_dir)

In [ ]:
output_basepath = output_root_dir + '/' + run_id + '_output'

for i in range(len(prompts)):
    if len(prompts) > 1:
        current_output = output_basepath + '.' + str(i) + output_ext
    else:
        current_output = output_basepath + output_ext
    ai.generate_to_file(n=num_outputs,
                        batch_size=1,
                        prompt=prompts[i],
                        max_length=max_length,
                        temperature=temperature,
                        top_p=top_p,
                        destination_path=current_output)

07/06/2022 19:04:28 — INFO — aitextgen — Generating 10 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43_output.0.txt


  0%|          | 0/10 [00:00<?, ?it/s]

07/06/2022 19:05:30 — INFO — aitextgen — Generating 10 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43_output.1.txt


  0%|          | 0/10 [00:00<?, ?it/s]

07/06/2022 19:06:51 — INFO — aitextgen — Generating 10 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43_output.2.txt


...

  0%|          | 0/10 [00:00<?, ?it/s]

07/06/2022 19:37:49 — INFO — aitextgen — Generating 10 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43_output.27.txt


  0%|          | 0/10 [00:00<?, ?it/s]

07/06/2022 19:39:05 — INFO — aitextgen — Generating 10 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43/aitextgen-base-gpt-neo-125M_run_2022-06-07-19-03-43_output.28.txt


  0%|          | 0/10 [00:00<?, ?it/s]

# LICENSE

MIT License

Copyright (c) 2020-2021 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.